# Homework 3 part 2

In [1]:
from collections import namedtuple

import numpy as np
from numpy.typing import NDArray
from tqdm import tqdm
from scipy.optimize import linprog

## Interior-point algorithm

In [12]:
Solution = namedtuple('Solution', ['x', 'y', 's', 'mu'])


def interior_point_step(
    A: NDArray, b: NDArray, c: NDArray, solution: Solution
) -> Solution:
    """Compute the next solution to problem:
        Ax = b
        A^Ty + s = c
        x > 0, s > 0
    with
        mu' = (1 - 1 / (6 * sqrt(m))) * mu

    Parameters:
        - A: constraints matrix
        - b: constraints vector
        - c: cost vector
        - solution: current solution (x, y, s, mu)

    Returns:
        - new solution (x', y', s', mu')
    """
    m = A.shape[1]
    mu = (1 - 1 / (6 * np.sqrt(m))) * solution.mu

    S_inv = np.diag(1 / solution.s)
    X = np.diag(solution.x)
    e = np.ones(m)

    # Solve system (S): (AS^(-1)XA^T) k = b - mu'AS^(-1)e
    k = np.linalg.solve(A @ S_inv @ X @ A.T, b - mu * A @ S_inv @ e)
    f = -A.T @ k
    h = -X @ S_inv @ f + mu * S_inv @ e - solution.x

    x = solution.x + h
    y = solution.y + k
    s = solution.s + f

    return Solution(x, y, s, mu)


def interior_point(A: NDArray, b: NDArray, c: NDArray, solution: Solution) -> Solution:
    """Solve the problem:
        min c^Tx
        Ax = b
        A^Ty + s = c
        x > 0, s > 0

    Parameters:
        - A: constraints matrix
        - b: constraints vector
        - c: cost vector
        - solution: initial solution (x0, y0, s0, mu0)

    Returns:
        - solution (x, y, s, mu)
    """
    with tqdm(desc=f'{c.dot(solution.x)}') as pbar:
        while True:
            pbar.update()

            new_solution = interior_point_step(A, b, c, solution)
            pbar.set_description(f'{c.dot(new_solution.x)}')

            if np.linalg.norm(new_solution.x - solution.x) < 1e-8:
                break
            solution = new_solution
    pbar.close()
    return solution

We define the linear program and the initial solution.

In [13]:
A = np.array([[3, 3, 3, 0, 0], [3, 1, 0, 1, 0], [1, 4, 0, 0, 1]])
c = np.array([-3, -4, 0, 0, 0])
b = np.array([4, 3, 4])

x = np.array([2 / 5, 8 / 15, 2 / 5, 19 / 15, 22 / 15])
y = np.array([-4 / 5, -4 / 5, -2 / 3])
s = np.array([37 / 15, 28 / 15, 12 / 5, 4 / 5, 2 / 3])
mu = 1

Check if the solution is strictly feasible.

In [14]:
print('Ax = b:', np.allclose(A @ x, b))
print('A^Ty + s = c:', np.allclose(A.T @ y + s, c))
print('x_i, s_j != 0', np.all(x > 1e-11) and np.all(s > 1e-11))

Ax = b: True
A^Ty + s = c: True
x_i, s_j != 0 True


In [15]:
# Mean of x * s works for mu
print('σ² =', np.sum((x * s / np.mean(x * s) - 1) ** 2))

print('σ² =', np.sum((x * s / mu - 1) ** 2))
print('σ² <= 1/4:', np.sum((x * s / mu - 1) ** 2) <= 1 / 4)

σ² = 0.0016232448664881226
σ² = 0.002469135802469144
σ² <= 1/4: True


In [16]:
solution = interior_point(A, b, c, Solution(x, y, s, mu))
print('x =', solution.x.round(5))
print('y =', solution.y.round(5))
print('s =', solution.s.round(5))
print('mu =', solution.mu)

-4.88888887655142: : 53it [00:00, 2189.37it/s]  

x = [0.44444 0.88889 0.      0.77778 0.     ]
y = [-0.88889 -0.      -0.33333]
s = [0.      0.      2.66667 0.      0.33333]
mu = 8.81247870897229e-09


### Analytical solution

We add equations:
- x_3 = 0
- x_5 = 0
- s_1 = 0
- s_2 = 0
- s_4 = 0

In [16]:
B = np.zeros((len(b) + len(c) + 5, len(x) + len(y) + len(s)))
# Ax = b
B[: len(b), : len(x)] = A
# A^Ty + s = c
B[len(b) : len(b) + len(c), len(x) : len(x) + len(y)] = A.T
B[len(b) : len(b) + len(c), len(x) + len(y) :] = np.eye(len(c))
# New equations for x and s
B[[np.arange(len(b) + len(c), B.shape[0])], [2, 4, 8, 9, 11]] = 1

d = np.concatenate([b, c, np.zeros(5)])

xys = np.linalg.solve(B, d)
print('x =', xys[: len(x)].round(5))
print('y =', xys[len(x) : len(x) + len(y)].round(5))
print('s =', xys[len(x) + len(y) :].round(5))

x = [ 0.44444  0.88889 -0.       0.77778  0.     ]
y = [-0.88889  0.      -0.33333]
s = [0.      0.      2.66667 0.      0.33333]


## Commercial solver

We use the `scipy`'s implementation of the simplex and interior-point method
to solve the linear program.

In [17]:
res_simplex = linprog(c, A_eq=A, b_eq=b, method='highs-ds')
print('x =', res_simplex.x)
print('Iterations:', res_simplex.nit)
print('Cost:', res_simplex.fun)

x = [0.44444444 0.88888889 0.         0.77777778 0.        ]
Iterations: 3
Cost: -4.888888888888889


In [18]:
res_interior_point = linprog(c, A_eq=A, b_eq=b, method='highs-ipm')
print('x =', res_interior_point.x)
print('Iterations:', res_interior_point.nit)
print('Cost:', res_interior_point.fun)

x = [0.44444444 0.88888889 0.         0.77777778 0.        ]
Iterations: 5
Cost: -4.888888888888889
